# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-14B 模型进行 LoRA 微调

### 1. 环境准备与库导入

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [2]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/miniconda3/envs/deepseek-quickstart/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [3]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-14B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.733 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.86s/it]


### 3. 微调前推理测试

In [4]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [5]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [6]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [7]:
print(response[0].split("### Response:")[1])


<think>
嗯，我现在要处理一个关于28岁程序员的健康问题。他最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。首先，我需要分析这些症状可能的原因。

首先，头晕和脖子疼可能与长时间的工作姿势有关。程序员通常久坐，可能姿势不良，导致肌肉疲劳或颈椎问题。这种情况下，可能诊断为颈部肌肉劳损或颈椎病。

其次，恶心可能与颈椎问题有关，因为颈椎病有时会引起椎动脉压迫，影响供血，导致头晕和恶心。也有可能是其他原因，比如血压异常、贫血或内耳问题，但结合他的工作习惯，颈椎问题更有可能。

接下来，我需要考虑其他可能的原因。比如，长时间使用电子设备可能导致眼睛疲劳，但通常不会引起严重的头晕和恶心。另外，压力大可能导致紧张性头痛，但这也通常不伴有恶心，除非是严重的偏头痛。

然后，我应该建议他进行一些初步的自我护理措施，比如改善工作姿势，适当休息，做一些颈部拉伸运动。同时，建议他减少电子设备的使用，保持良好的生活习惯，如规律作息和适量运动。

如果这些症状持续或加重，可能需要进一步的医疗评估。比如，进行颈椎的X光或MRI检查，排除椎间盘突出或其他结构性问题。同时，可能需要检查血压和血糖，排除糖尿病或高血压的可能性。

在治疗方面，物理治疗可能有助于缓解颈部肌肉紧张，改善血液循环。如果有必要，可以考虑药物治疗，如非甾体抗炎药来缓解疼痛和炎症。此外，心理压力管理也是重要的一环，可能建议他进行放松训练或寻求心理辅导。

总的来说，这个病例主要考虑颈椎问题的可能性较大，但也不能完全排除其他原因，因此建议他及时就医进行详细检查，以获得准确的诊断和适当的治疗。
</think>




### 4. 下载和格式化训练数据集

医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

In [8]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [9]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [10]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [11]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

In [12]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
        (layers): ModuleList(
          (0-4): 5 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=5120, out_features=5120, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

### 6. 配置 SFTTrainer

In [13]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Detected kernel version 5.4.119, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### 7. 开始训练

In [14]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 2 x 1) = 64
 "-____-"     Trainable parameters = 68,812,800 of 14,838,846,464 (0.46% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 28 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (28 x 2 x 1) = 56
 "-____-"     Trainable parameters = 68,812,800 of 14,838,846,464 (0.46% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 25 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.707900
2,2.654800
3,2.639700
4,2.491600
5,2.472700
6,2.320500
7,2.256600
8,2.177500
9,2.042200
10,2.016600


TrainOutput(global_step=60, training_loss=1.7653941671053568, metrics={'train_runtime': 1746.496, 'train_samples_per_second': 0.893, 'train_steps_per_second': 0.034, 'total_flos': 1.1465029713993523e+17, 'train_loss': 1.7653941671053568})


### 8. 保存微调后的模型（Lora）

In [23]:
model.save_pretrained("qwen-14b_lora_model")

In [24]:
tokenizer.save_pretrained("qwen-14b_lora_model")

('qwen-14b_lora_model/tokenizer_config.json',
 'qwen-14b_lora_model/special_tokens_map.json',
 'qwen-14b_lora_model/chat_template.jinja',
 'qwen-14b_lora_model/tokenizer.json')

In [25]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-14b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [26]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [27]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
嗯，这位病人已经患急性阑尾炎5天了，腹痛虽然稍微减轻，但还是发烧呢。这可不太寻常，通常急性阑尾炎在发病24小时内会开始好转，但这位病人却在5天后还在发烧，说明可能有并发症。再想想，阑尾炎如果长时间不处理，可能会发展成阑尾穿孔，这样可就麻烦了。

哦，对了，病人在体检时发现右下腹有压痛的包块。这可是个危险的信号，很可能阑尾已经穿孔了，导致腹膜炎。腹膜炎的话，需要立即处理，否则会很危险。

所以，这个时候，我觉得应该赶紧安排手术。手术是处理急性阑尾炎并发症最直接的办法。当然，手术前还得做些准备，比如完善一些检查，比如血常规、腹部超声，看看情况到底有多严重。

不过，手术之前，是不是得先看看有没有其他可能性呢？比如阑尾炎穿孔之前有没有其他症状，或者是不是其他地方出了问题。这些检查有助于确认诊断，确保手术的必要性。

如果确实诊断为阑尾穿孔，那手术就变得很紧迫了。毕竟，腹膜炎如果不及时处理，很快就会危及生命。

当然，手术后还得注意恢复。术后得好好护理，防止感染，让病人尽快康复。总之，面对这种复杂情况，及时而准确的处理是关键。
</think>
在这种情况下，病人已经患急性阑尾炎5天，腹痛稍有减轻但仍然发热，且在体检时发现右下腹有压痛的包块。这些症状提示可能存在阑尾穿孔导致的腹膜炎。

在这种情况下，应该立即安排手术治疗，以防止病情进一步恶化。手术是处理急性阑尾炎并发症最直接的方法。在手术前，建议进行一些必要的检查，如血常规和腹部超声，以确认诊断，确保手术的必要性。

如果确实诊断为阑尾穿孔，手术就变得非常紧迫，因为腹膜炎如果不及时处理，会迅速危及生命。手术后，需要严格遵守术后护理措施，防止感染，并帮助病人尽快康复。

因此，面对这种情况，及时和准确的处理是关键，以确保病人的安全和健康。



In [28]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [29]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这个患者60岁，是男性，他出现了右侧胸疼，而且在X线检查中发现右侧肋膈角消失了。这让我想到，他可能有胸腔积液，而且X线显示肋膈角消失，这通常提示有积液，因为积液会把肋膈角给遮盖了。

所以，先来想想，这可能是什么原因引起的。胸腔积液的原因有很多种，比如肺结核、肺炎、心力衰竭、肿瘤等。不过，结合患者的年龄和胸腔积液，肺结核是一个比较常见的可能性。

现在，我们需要通过实验室检查来进一步明确胸水的性质。通常，胸水的性质可以通过一些特定的实验室检查来判断，比如胸水常规分析、生化检查，以及细菌学检查等。

首先，胸水常规分析可以帮助我们了解胸水的细胞数目、颜色、粘稠度等基本信息。这些信息可以帮助我们初步判断胸水的类型，比如是渗出液还是漏出液，还有细胞类型。

不过，对于肺结核来说，胸水通常会有一些特点，比如细胞数目可能增多，细胞分类中可能以淋巴细胞为主。但是，单凭常规分析可能还不够，我们需要更多的信息来确认。

接下来，生化检查可以进一步提供线索。比如，胸水中的蛋白含量、乳酸脱氢酶（LDH）水平、pH值等。这些指标可以帮助我们判断胸水的性质，比如是炎症性还是感染性。

对于肺结核，胸水的蛋白含量通常会升高，而LDH水平也可能升高，这提示胸水可能与炎症或感染有关。此外，胸水的pH值通常会降低，这可能与感染或炎症有关。

不过，这些信息虽然有帮助，但还不能完全确定胸水的性质，尤其是对于肺结核来说，我们需要更具体的证据。

再来看细菌学检查，比如胸水的细菌培养和抗酸染色。如果胸水中有结核杆菌，那么这些检查会显示阳性，这可以确认是肺结核导致的胸水。

但是，细菌培养需要时间，而且阳性率可能不是百分之百，尤其是当胸水中的结核杆菌数量较少时。

所以，综合来看，为了快速且有效地确认胸水的性质，尤其是对于肺结核，我们可能需要结合其他检查。比如，胸水的抗酸染色可以快速检测结核杆菌的存在，这对于确认胸水的性质非常有帮助。

总的来说，尽管胸水常规分析和生化检查提供了初步的信息，但胸水的抗酸染色可能在确认胸水性质方面更有帮助，尤其是在肺结核的诊断中。
</think>
对于这名60岁男性患者，右侧胸疼和X线检查显示右侧肋膈角消失，初步考虑为肺结核伴右侧胸腔积液。要确认胸水的性质，尤

In [30]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这个28岁的程序员，他常年熬夜，工作压力也大，突然头晕目眩，还有点恶心，这可不太寻常。首先，我得考虑一下他的生活习惯。熬夜肯定是个大问题，长期睡眠不足可能会影响他的身体机能，导致血压异常或者精神状态不佳。

再想想，程序员通常久坐不动，这对血液循环可不太好。突然站起来头晕，这让我想到可能是低血压，或者是体位性低血压。低血压的话，通常是因为血液循环不好，导致身体供血不足，尤其是在站起的时候，这会让人感觉头晕。

还有，他的恶心症状让我想到可能是中暑，但通常中暑是夏天在高温下才会出现，而他现在的情况可能不是中暑。也有可能是内耳的问题，比如梅尼埃病，这种病会导致头晕和恶心，但通常会有听力下降，而他并没有提到听力问题。

再想想，他可能还有颈椎病，因为长期的久坐和不正确的坐姿，都可能导致颈椎问题，这样头晕和恶心也是有可能的。不过，通常颈椎病会有更多的症状，比如颈部僵硬或者放射性疼痛，他并没有提到这些。

还有，他最近有没有什么情绪上的问题呢？压力大、工作紧张可能会引发焦虑，而焦虑有时候也会导致头晕和恶心。嗯，这似乎是个可能的原因。

再想想，他的头晕症状是不是在站起的时候更明显？如果是这样，那可能就是体位性低血压。这种情况下，人站起时血压突然下降，导致供血不足，就会头晕甚至晕厥。

综合来看，他的症状更符合低血压或者体位性低血压，再加上他长期熬夜和久坐的工作习惯，这些都可能是导致这些症状的原因。当然，最好还是去检查一下血压，看看是不是血压真的低，或者有没有其他潜在的问题。

如果检查后发现血压正常，那可能需要考虑其他可能性，比如内耳问题或者颈椎病，或者压力引起的问题。总之，低血压是一个比较合理的解释，但也不能排除其他可能。
</think>
根据提供的症状，28岁的男性程序员突然出现头晕目眩、甚至恶心，这种情况可能与以下因素有关：

1. **低血压或体位性低血压**：长期熬夜和久坐可能导致血液循环不良，体位性低血压在站起时会因为血压突然下降而导致头晕和恶心。

2. **颈椎病**：由于程序员的工作特点，长期久坐可能导致颈椎问题，进而引发头晕和恶心。

3. **压力和焦虑**：工作压力大、睡眠不足可能引起焦虑，而焦虑有时也会导致头晕和恶心。

4. **生活

In [31]:
### 10.加载 1.5b sft后的模型进行对比

In [33]:
# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
sft_model_name_1_dot_5_b = 'qwen-1.5b_lora_model'
model_sft_1_dot_5_b, tokenizer_sft_1_dot_5_b = FastLanguageModel.from_pretrained(
    model_name = sft_model_name_1_dot_5_b,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.733 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [34]:
# 对比
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model_sft_1_dot_5_b, tokenizer_sft_1_dot_5_b, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这位60岁的男性患者，胸疼，X线显示右侧肋膈角消失，这让我想到可能有肺结核。肺结核通常会导致胸腔积液，尤其是在右侧。现在，我得想想，要了解胸水的性质，应该怎么做呢？

首先，胸水的性质可能和肺结核的类型有关。如果是肺结核，通常会有肺部的结核菌，这些菌会在胸腔内繁殖，导致积液。如果肺结核是原发性，那么胸腔内的结核菌也会繁殖，这可能影响胸水的性质。

那，我需要什么检测能帮助我了解这个呢？嗯，如果我需要了解胸水的类型，比如是否是结核菌导致的，那我得用抗结核素检测。这可以帮助我判断是肺结核还是其他类型的结核病。

如果要了解胸水的结构，比如是否是结核菌引起的，那可能需要一些超声或CT扫描，这样我可以检查一下胸腔内的结构。

不过，最直接的方法还是得先知道是否是肺结核。因为肺结核的胸腔积液通常会引发胸水的性质变化。所以，抗结核素检测应该是关键，它可以帮助我判断是肺结核还是其他类型的结核病。

当然，如果怀疑不是肺结核，但有其他原因导致胸腔积液，我还需要进一步检查，比如血常规、胸片或CT，以确认病因。

总的来说，抗结核素检测是最直接的，因为它能帮助我们了解胸水的性质，从而指导进一步的检查和诊断。所以，这个过程应该先做抗结核素检测。
</think>
对于这位60岁男性患者，若X线检查显示右侧肋膈角消失并诊断为肺结核伴右侧胸腔积液，抗结核素检测是最直接的检查方法，它可以提供关于胸腔积液的性质和类型的信息。如果抗结核素检测结果表明是肺结核，那么胸腔积液可能与肺部结核菌有关，这种情况下，胸腔积液的性质可能与肺结核菌有关。相反，若检测结果不明确，可能需要进一步的检查以确定病因。因此，抗结核素检测是了解胸水性质的最有效手段。


In [35]:
# 对比
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model_sft_1_dot_5_b, tokenizer_sft_1_dot_5_b, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
哦，这个28岁的男性患者，工作是程序员，经常熬夜，最近又出现了头晕目眩和恶心的症状。嗯，这些症状让我想到可能是脑力减退吧，毕竟程序员经常熬夜，长期工作压力大。

脑力减退通常和脑损伤有关，尤其是脑膜炎。脑膜炎是脑部的炎症，通常会导致疲劳、恶心和头晕，这些症状正是我们看到的。不过，脑膜炎也有例外，比如脑膜炎的某些并发症，比如脑膜炎性脑水肿，也会导致一些类似的症状。

现在，我需要考虑脑膜炎和脑膜炎性脑水肿的可能性。脑膜炎性脑水肿是由于脑膜炎引发的神经元损伤，导致血液中的神经冲动被抑制，从而引起一些全身性的症状，比如恶心、头晕和肌肉无力，这些症状也符合这位患者的情况。

不过，再仔细想想，脑膜炎通常会伴随一些神经功能障碍，比如中枢神经系统功能减退，但是这些症状主要是由于脑膜炎引起的，而不仅仅是脑膜炎性脑水肿。

脑膜炎性脑水肿是更典型的脑膜炎并发症，它通常与脑膜炎一起出现，但并不是由脑膜炎单独引发的。因此，虽然脑膜炎是可能的原因之一，但脑膜炎性脑水肿更符合这位患者的情况。

综上所述，这位患者最可能的疾病是脑膜炎性脑水肿。当然，也有可能是脑膜炎本身，但脑膜炎性脑水肿更符合患者的症状和情况。
</think>
这位28岁的男性患者，工作是程序员，长期熬夜，最近出现头晕目眩和恶心等症状，最可能的疾病是脑膜炎性脑水肿。脑膜炎通常会导致脑部的炎症，从而引起头晕目眩和恶心等全身性症状。脑膜炎性脑水肿则是由于脑膜炎引发的神经元损伤，导致血液中的神经冲动被抑制，从而引起一些全身性的症状，如恶心、头晕和肌肉无力。因此，脑膜炎性脑水肿是最符合患者症状和情况的疾病。
